# Inference

In [1]:
# imports
import ast
import os
import pathlib

import neptune
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.models.detection.transform import GeneralizedRCNNTransform

from scripts.datasets import ObjectDetectionDatasetSingle, ObjectDetectionDataSet
from scripts.faster_RCNN import get_fasterRCNN_resnet
from scripts.transformations import ComposeDouble
from scripts.transformations import ComposeSingle
from scripts.transformations import FunctionWrapperDouble
from scripts.transformations import FunctionWrapperSingle
from scripts.transformations import apply_nms, apply_score_threshold
from scripts.transformations import normalize_01
from scripts.utils import get_filenames_of_path, collate_single, save_json
from scripts.visual import DatasetViewer
from scripts.visual import DatasetViewerSingle
from scripts.balayage import *
from skimage.io import imread

In [11]:
# parameters
params = {'INPUT_DIR': 'Input',  # files to predict
          'PREDICTIONS_PATH': 'Output',  # where to save the predictions
          'MODEL_DIR': 'model/checkpoint.ckpt',  # load model from checkpoint
          }
parameters = {'BATCH_SIZE': 2,
          'OWNER': 'bluestone',  # set your name here, e.g. johndoe22
          'SAVE_DIR': None,  # checkpoints will be saved to cwd
          'GPU': 1,  # set to None for cpu training
          'LR': 0.001,
          'PRECISION': 32,
          'CLASSES': 2,
          'SEED': 42,
          'PROJECT': 'Heads',
          'EXPERIMENT': 'heads',
          'MAXEPOCHS': 500,
          'BACKBONE': 'resnet34',
          'FPN': 'False',
          'ANCHOR_SIZE': '((32, 64, 128, 256, 512),)',
          'ASPECT_RATIOS': '((0.5, 1.0, 2.0),)',
          'MIN_SIZE': 1024,
          'MAX_SIZE': 10000,
          'IMG_MEAN': '[0.485, 0.456, 0.406]',
          'IMG_STD': '[0.229, 0.224, 0.225]',
          'IOU_THRESHOLD': 0.5
          }

In [3]:
# input files
inputs = get_filenames_of_path(pathlib.Path(params['INPUT_DIR']))
inputs.sort()

In [4]:
# transformations
transforms = ComposeSingle([
    FunctionWrapperSingle(np.moveaxis, source=-1, destination=0),
    FunctionWrapperSingle(normalize_01)
])

In [5]:
checkpoint = torch.load(params['MODEL_DIR'])
model_state_dict = checkpoint['hyper_parameters']['model'].state_dict()

In [6]:
# model init
model = get_fasterRCNN_resnet(num_classes=int(parameters['CLASSES']),
                              backbone_name=parameters['BACKBONE'],
                              anchor_size=ast.literal_eval(parameters['ANCHOR_SIZE']),
                              aspect_ratios=ast.literal_eval(parameters['ASPECT_RATIOS']),
                              fpn=ast.literal_eval(parameters['FPN']),
                              min_size=int(parameters['MIN_SIZE']),
                              max_size=int(parameters['MAX_SIZE'])
                              )

In [7]:
# load weights
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [21]:
# inference (cpu)
model.eval()
for filename in inputs : 
    x = imread(filename)
    x=np.array(x)
    dic = balayage_inference_single(x,(1024,1024), 512)
    preds = inference_on_balayage(dic,model,transforms)    
    save_json(preds, path=pathlib.Path('Output/'+ os.path.basename(filename)).with_suffix('.json'))
    print('saved file for image: '+str(filename))

(1272, 1024)
(1024, 1080)
processing image: 0
processing image: 0 DONE
processing image: 1
processing image: 1 DONE
processing image: 2
processing image: 2 DONE
processing image: 3
processing image: 3 DONE
processing image: 4
processing image: 4 DONE
processing image: 5
processing image: 5 DONE
processing image: 6
processing image: 6 DONE
processing image: 7
processing image: 7 DONE
processing image: 8
processing image: 8 DONE
processing image: 9
processing image: 9 DONE
processing image: 10
processing image: 10 DONE
processing image: 11
processing image: 11 DONE
processing image: 12
processing image: 12 DONE
processing image: 13
processing image: 13 DONE
processing image: 14
processing image: 14 DONE
processing image: 15
processing image: 15 DONE
processing image: 16
processing image: 16 DONE
processing image: 17
processing image: 17 DONE
processing image: 18
processing image: 18 DONE
processing image: 19
processing image: 19 DONE
processing image: 20
processing image: 20 DONE
process

In [12]:
# get prediction files
predictions = get_filenames_of_path(pathlib.Path('Inférence/Output'))
predictions.sort()

In [13]:
# create prediction dataset
iou_threshold = 0.25
score_threshold = 0.6

transforms_prediction = ComposeDouble([
    FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01),
    FunctionWrapperDouble(apply_nms, input=False, target=True, iou_threshold=iou_threshold),
    FunctionWrapperDouble(apply_score_threshold, input=False, target=True, score_threshold=score_threshold)
])

dataset_prediction = ObjectDetectionDataSet(inputs=inputs,
                                            targets=predictions,
                                            transform=transforms_prediction,
                                            use_cache=False)

In [14]:
# mapping
color_mapping = {
    1: 'red',
}

In [15]:
# visualize predictions
datasetviewer_prediction = DatasetViewer(dataset_prediction, color_mapping)
datasetviewer_prediction.napari()
# add text properties gui
datasetviewer_prediction.gui_text_properties(datasetviewer_prediction.shape_layer)